In [ ]:
import pyaudio as pa

p = pa.PyAudio()

print('Available audio input devices:')
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    if dev.get('maxInputChannels'):
        print(i, dev.get('name'))
print('Please type input device ID:')
dev_idx = int(input())

In [ ]:
from frame_asr import FrameASR
asr = FrameASR()

In [ ]:
import numpy as np
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
import pyaudio as pa
import time

CHANNELS = 1
RATE = 16000
DURATION = 2.0
CHUNK_SIZE = int(DURATION*RATE)

output_notebook()

fig = figure(plot_width=800, plot_height=400)
signal = np.zeros(CHUNK_SIZE)
source = {'x': np.arange(len(signal)), 'y': signal}
plot = fig.line('x', 'y', source=source)
show(fig, notebook_handle=True)


def callback(in_data, frame_count, time_info, status):
    signal = np.frombuffer(in_data, dtype=np.int16)
    plot.data_source.data['y'] = signal
    push_notebook()
    pred = asr.transcribe(signal)
    if len(pred.strip()):
        print('"{}"'.format(pred))
    return (in_data, pa.paContinue)



stream = p.open(format=pa.paInt16,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                input_device_index=dev_idx,
                stream_callback=callback,
                frames_per_buffer=CHUNK_SIZE)

stream.start_stream()

while stream.is_active():
    time.sleep(0.1)

stream.stop_stream()
stream.close()

p.terminate()